In [2]:
from Tool_Functions.cleaning_data import *
from Tool_Functions.comportment_reabo import * 
from Tool_Functions.join_data import * 

In [3]:
path_antoine = "/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/"

Nouveaux indicateurs : 
- Mois du réabonnement 
- Delai de réabonnement moyen pour cette promo comparé au délai de réabonnement moyen pour toutes les promos
- Trouver aussi un moyen de quantifier le poids de l’abonnement par rapport aux habitudes de fidélité d’un client : s’il s’abonne alors qu’il s’abonne jamais, la promo aura un poids plus important

•⁠  ⁠transformer la fidélité en un score et essayer de le croiser avec les types de promo
•⁠  ⁠⁠comparer les délais de réabonnements à ce qu’il se fait hors promotion 
•⁠  ⁠⁠on peut dans un premier temps se concentrer sur un seul type d’ODD (ex 15j TC)

In [1]:
def calculer_score_fidélité(abonnements):
    """
    Calcule le score de fidélité en fonction des abonnements effectués.
    
    Args:
    abonnements : list[int] : Liste des nombres d'abonnements effectués au cours des 12 mois glissants.

    Returns:
    int : Score de fidélité.
    """
    # Si l'abonné ne s'est jamais abonné, retourner un score minimal
    if not abonnements:
        return 0
    
    # Calculer le score comme la somme des abonnements sur la période
    score = sum(abonnements)
    
    # Nombre total d'abonnements sur la période
    nombre_abonnements = len(abonnements)
    
    # Score maximal possible est lorsque l'abonné s'est abonné chaque mois
    score_maximal = 13 * nombre_abonnements
    
    # Normaliser le score entre 0 et 100
    score_normalisé = (score / score_maximal) * 100
    
    # Ponderation du score en fonction de la tendance de la fidélité récente
    poids_fidélité_récente = 0.6
    dernier_abonnement = abonnements[-1]
    
    # Score ajusté en fonction du nombre de chiffres dans la liste des abonnements
    score_ajusté = score_normalisé * (1 + 0.1 * (nombre_abonnements - 1))
    
    # Score final
    score_final = (score_ajusté * poids_fidélité_récente) + (dernier_abonnement * (1 - poids_fidélité_récente))
    
    return round(score_final)

In [8]:
def clean_nombres_abonnements(abonnements):
    """
    Convertit les valeurs de la colonne 'NOMBRE_ABONNEMENTS' en listes d'entiers.
    """
    try:
        return eval(abonnements)
    except:
        return []

In [9]:
def colonne_score(data_path):
    df = file_to_dataframe(data_path + "fusion_table_v3.csv")
    df['NOMBRE_ABONNEMENTS'] = df['NOMBRE_ABONNEMENTS'].apply(clean_nombres_abonnements)
    df['SCORE_FIDELITE'] = df['NOMBRE_ABONNEMENTS'].apply(calculer_score_fidélité)
    save_to_csv_file(df, path_antoine + 'new_fusion_table_score.csv')
    return True

In [10]:
colonne_score(path_antoine)

True

In [11]:
df = file_to_dataframe(path_antoine + 'new_fusion_table_score.csv') 

In [22]:
test

,ID_ABONNE,Autres_n_REABOS,ODD 15 jours EV+_n_REABOS,ODD 15 jours TC_n_REABOS,ODD 21 jours TC_n_REABOS,ODD 30 jours EV+_n_REABOS,ODD 30 jours TC_n_REABOS,ODD 7 jours autre que SG_n_REABOS,PAS_ODD_n_REABOS,Semaine genéreuse_n_REABOS,...,5,6,7,8,9,10,11,12,NOMBRE_ABONNEMENTS,SCORE_FIDELITE
21713,50576762801,0,0,6,0,0,0,0,0,9,...,2,0,2,0,1,0,2,0,[15],75
63345,51024205801,0,0,5,0,0,0,0,0,10,...,1,1,1,1,1,1,2,1,[15],75
66223,51036157101,1,0,4,0,2,1,0,0,19,...,0,3,0,3,2,4,1,3,"[12, 15]",75
78830,51103353201,0,0,5,0,0,0,0,0,10,...,1,1,1,1,2,0,1,2,[15],75
94230,51161104401,0,0,7,0,1,0,0,0,19,...,2,1,1,5,5,1,2,0,"[11, 16]",75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
666741,52647378401,0,1,4,0,1,1,0,1,7,...,3,0,3,2,1,0,1,1,[15],75
669163,52655068201,0,1,2,0,0,1,0,0,11,...,2,0,2,1,2,1,1,2,[15],75
673054,52667783501,0,1,5,0,0,1,0,0,8,...,2,1,1,1,2,1,1,2,[15],75
673893,52671045101,0,1,6,0,0,0,0,0,8,...,2,1,1,1,2,1,1,1,[15],75


In [26]:
def identifier_anticipation(abonnements):
    """
    Identifie les abonnés qui ont anticipé en se réabonnant plusieurs fois dans le même mois.
    Par exemple, si le nombre d'abonnements dépasse un certain seuil, nous pouvons considérer que l'abonné a anticipé.
    """
    seuil_anticipation = 14  # Seuil à ajuster selon vos critères
    if max(abonnements) >= seuil_anticipation:
        return True
    else:
        return False


In [27]:

def recalibrer_score(abonnements, score_maximal):
    """
    Recalibre le score de fidélité pour le groupe des abonnés qui n'ont pas anticipé.
    """
    score = sum(abonnements)
    score_normalisé = (score / score_maximal) * 100
    return round(score_normalisé)


In [32]:
def colonne_score_v2(data_path):
    df = pd.read_csv(data_path + "fusion_table_score.csv")
    
    # Nettoyer la colonne 'NOMBRE_ABONNEMENTS'
    df['NOMBRE_ABONNEMENTS'] = df['NOMBRE_ABONNEMENTS'].apply(clean_nombres_abonnements)
    
    # Identifier les abonnés qui ont anticipé
    df['ANTICIPATION'] = df['NOMBRE_ABONNEMENTS'].apply(identifier_anticipation)
    
    # Définir un score maximal pour le recalibrage
    score_maximal_rec = 13 * len(df['NOMBRE_ABONNEMENTS'].iloc[0])
    
    # Calculer le score pour chaque groupe
    df['SCORE_FIDELITE_2'] = df.apply(lambda row: recalibrer_score(row['NOMBRE_ABONNEMENTS'], score_maximal_rec) if not row['ANTICIPATION'] else 100, axis=1)
    
    # Sauvegarder le dataframe avec le score de fidélité dans un nouveau fichier CSV
    df.to_csv(data_path + 'new_fusion_table_score_2.csv', index=False)
    
    return True

In [33]:
colonne_score_v2(path_antoine)

True

In [34]:
df2 = file_to_dataframe(path_antoine + 'new_fusion_table_score_2.csv')

In [35]:
df2

,ID_ABONNE,Autres_n_REABOS,ODD 15 jours EV+_n_REABOS,ODD 15 jours TC_n_REABOS,ODD 21 jours TC_n_REABOS,ODD 30 jours EV+_n_REABOS,ODD 30 jours TC_n_REABOS,ODD 7 jours autre que SG_n_REABOS,PAS_ODD_n_REABOS,Semaine genéreuse_n_REABOS,...,6,7,8,9,10,11,12,NOMBRE_ABONNEMENTS,ANTICIPATION,SCORE_FIDELITE_2
0,50402221602,0,0,1,0,0,2,0,1,0,...,1,1,0,0,0,0,2,"[3, 1]",False,15
1,50402223202,0,0,8,0,0,4,0,0,17,...,2,1,2,2,3,2,2,"[11, 9, 9]",False,112
2,50402224003,0,0,3,0,0,0,0,0,4,...,1,0,0,0,0,0,1,[7],False,27
3,50402230202,0,0,9,0,0,3,0,0,6,...,2,1,1,1,2,1,2,"[8, 10]",False,69
4,50402230203,0,0,11,1,0,3,0,0,5,...,1,1,1,1,2,1,2,"[8, 12]",False,77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
780791,53186931901,0,1,1,0,0,0,0,0,1,...,0,0,0,2,1,0,0,[3],False,12
780792,53186937201,0,1,1,0,0,0,0,0,1,...,0,0,0,2,1,0,0,[3],False,12
780793,53186971001,0,1,1,0,0,0,0,0,1,...,0,0,0,2,1,0,0,[3],False,12
780794,53186977101,0,1,1,0,0,0,0,0,1,...,0,0,0,2,1,0,0,[3],False,12


In [36]:
def colonne_score_v3(data_path):
    df = pd.read_csv(data_path + "new_fusion_table_score.csv")
    
    # Nettoyer la colonne 'NOMBRE_ABONNEMENTS'
    df['NOMBRE_ABONNEMENTS'] = df['NOMBRE_ABONNEMENTS'].apply(clean_nombres_abonnements)
    
    # Identifier les abonnés qui ont anticipé
    df['ANTICIPATION'] = df['NOMBRE_ABONNEMENTS'].apply(identifier_anticipation)
    
    # Définir un score maximal pour le recalibrage
    score_maximal_rec = 13 * len(df['NOMBRE_ABONNEMENTS'].iloc[0])
    
    # Calculer le score pour chaque groupe
    df['SCORE_FIDELITE_2'] = df.apply(lambda row: recalibrer_score(row['NOMBRE_ABONNEMENTS'], score_maximal_rec) if not row['ANTICIPATION'] else 100, axis=1)
    
    # Sauvegarder le dataframe avec le score de fidélité dans un nouveau fichier CSV
    df.to_csv(data_path + 'new_fusion_table_score_comp.csv', index=False)
    
    return True

In [37]:
colonne_score_v3(path_antoine)

True

In [41]:
df3 = file_to_dataframe(path_antoine + 'new_fusion_table_score_comp.csv')

In [40]:
df3['SCORE_FIDELITE_2'].unique()

array([ 15, 112,  27,  69,  77,  92,  62, 119,  23, 115,  81,  65,  50,
        12, 123,  85, 100,  38, 127,  19, 104,  88,  35,  46, 108,  58,
        31,  96,  73,  42,  54])

In [42]:
df3

,ID_ABONNE,Autres_n_REABOS,ODD 15 jours EV+_n_REABOS,ODD 15 jours TC_n_REABOS,ODD 21 jours TC_n_REABOS,ODD 30 jours EV+_n_REABOS,ODD 30 jours TC_n_REABOS,ODD 7 jours autre que SG_n_REABOS,PAS_ODD_n_REABOS,Semaine genéreuse_n_REABOS,...,7,8,9,10,11,12,NOMBRE_ABONNEMENTS,SCORE_FIDELITE,ANTICIPATION,SCORE_FIDELITE_2
0,50402221602,0,0,1,0,0,2,0,1,0,...,1,0,0,0,0,2,"[3, 1]",11,False,15
1,50402223202,0,0,8,0,0,4,0,0,17,...,1,2,2,3,2,2,"[11, 9, 9]",57,False,112
2,50402224003,0,0,3,0,0,0,0,0,4,...,0,0,0,0,0,1,[7],35,False,27
3,50402230202,0,0,9,0,0,3,0,0,6,...,1,1,1,2,1,2,"[8, 10]",50,False,69
4,50402230203,0,0,11,1,0,3,0,0,5,...,1,1,1,2,1,2,"[8, 12]",56,False,77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
780791,53186931901,0,1,1,0,0,0,0,0,1,...,0,0,2,1,0,0,[3],15,False,12
780792,53186937201,0,1,1,0,0,0,0,0,1,...,0,0,2,1,0,0,[3],15,False,12
780793,53186971001,0,1,1,0,0,0,0,0,1,...,0,0,2,1,0,0,[3],15,False,12
780794,53186977101,0,1,1,0,0,0,0,0,1,...,0,0,2,1,0,0,[3],15,False,12
